In [1]:
input_data = 'final_data.csv'
output_data = 'prediction_20191014.csv'

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
import glob

In [3]:
kfold = StratifiedKFold(n_splits=5) # 하이퍼 파라미터 지정
n_it = 12

In [ ]:
np.random.seed(123)
main= pd.read_csv(input_data)
train = main[main.GENDER!='UNKNOWN']
train.GENDER = (train.GENDER=='M').astype(int)

test = main[main.GENDER=='UNKNOWN']
test = test.sort_values('MEM_ID')

t_final = test[['MEM_ID', 'GENDER']]

test = test.drop(['GENDER','MEM_ID'], axis=1)
target = train.GENDER.values
train = train.drop(['GENDER','MEM_ID'], axis=1)

params = {'max_features':list(np.arange(1, train.shape[1])), 'bootstrap':[False], 'n_estimators': [50], 'criterion':['gini','entropy']}
model = RandomizedSearchCV(RandomForestClassifier(), param_distributions=params, n_iter=n_it, cv=kfold, scoring='roc_auc',n_jobs=-1, verbose=1)
print('MODELING.............................................................................')
model.fit(train, target)
print('========BEST_AUC_SCORE = ', model.best_score_)
model = model.best_estimator_
t_final.GENDER = model.predict_proba(test.values)[:,1]
t_final.to_csv(output_data, index=False)
print('COMPLETE')

c:\users\kduar_000\anaconda3\lib\site-packages\pandas\core\generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


MODELING.............................................................................
Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.2min
